In [1]:
!pip install faster-whisper
!pip install jiwer
!pip install pydub

  Using cached faster_whisper-1.0.1-py3-none-any.whl.metadata (14 kB)
  Using cached av-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached ctranslate2-4.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached onnxruntime-1.17.3-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
Using cached faster_whisper-1.0.1-py3-none-any.whl (1.5 MB)
Using cached av-11.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32.9 MB)
Using cached ctranslate2-4.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.7 MB)
Using cached huggingface_hub

In [2]:
import os

# The directory where libcublas.so.12 is located
cublas_directory = '/opt/conda/lib/python3.10/site-packages/nvidia/cublas/lib'

# Get the current LD_LIBRARY_PATH
current_ld_library_path = os.environ.get('LD_LIBRARY_PATH', '')

# If the cublas directory is not already in LD_LIBRARY_PATH, add it
if cublas_directory not in current_ld_library_path:
    os.environ['LD_LIBRARY_PATH'] = cublas_directory + ':' + current_ld_library_path

# Print the updated LD_LIBRARY_PATH to verify the change
print("Updated LD_LIBRARY_PATH:", os.environ['LD_LIBRARY_PATH'])


Updated LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/nvidia/cublas/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [3]:
import ctypes
import os

# Path to the libcublas library
libcublas_path = '/opt/conda/lib/python3.10/site-packages/nvidia/cublas/lib/libcublas.so.12'

# Try loading the library directly
try:
    ctypes.CDLL(libcublas_path)
    print("Library loaded successfully!")
except OSError as e:
    print("Failed to load library:", str(e))


Library loaded successfully!


In [4]:
import os

# Print environment variables related to library paths
print("LD_LIBRARY_PATH:", os.environ.get('LD_LIBRARY_PATH'))
print("PATH:", os.environ.get('PATH'))


LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/nvidia/cublas/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
PATH: /opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [18]:
import time
from faster_whisper import WhisperModel
import faster_whisper
from jiwer import wer, cer
model_size = "large-v3"

# Initialize WhisperModel for fast processing on appropriate hardware
model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# model = faster_whisper.WhisperModel("large-v3")

In [26]:
# import time
# from faster_whisper import WhisperModel
# from jiwer import wer, cer

# model_size = "large-v3"

# Initialize WhisperModel
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

# Run on GPU with FP16
# model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")

# Define the audio file path
audio_file = "Whisper.mp3"

# Reference sentence
reference_text = "The Whisper models are trained for speech recognition and translation tasks. Capable of transcribing speech audio into the text in the language it is spoken, which is also known as automatic speech recognition, as well as translated into speech, especially in English, which is speech translation. Researchers at OpenAI developed the model to study the robustness of speech processing systems trained under large-scale weak supervision. There are nine models of different sizes and capabilities summarized in the following table. According to the size, the tiny model has 39 million parameters and it is English only. Base model has 74 million parameters and it is also English only model. Then there is a small model, which has around 244 million parameters and it is also English only. Then there is a medium model. The medium model has 796 million parameters and it is English only model as well. Then there is a large model as well, which has 100, sorry, it has 1550 million parameters and it is a multilingual model. Thank you. "

# Transcribe audio and measure latency
start_time = time.time()
segments, info = model.transcribe(audio_file, beam_size=5)
end_time = time.time()
latency = end_time - start_time

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

# Process segments and compute WER and CER
transcribed_text = " ".join(segment.text for segment in segments)
wer_result = wer(reference_text, transcribed_text)
cer_result = cer(reference_text, transcribed_text)

# Print results
print("Transcribed Text:", transcribed_text)
print("WER:", wer_result)
print("CER:", cer_result)
print("Latency:", latency, "seconds")

Detected language 'en' with probability 0.985840
Transcribed Text:  The WISPR models are trained for speech recognition and translation tasks.  Capable of transcribing speech audio into the text in the language it is spoken,  which is also known as automatic speech recognition,  as well as translated into speech, especially in English, which is speech translation.  Researchers at OpenAI developed the model to study the robustness of speech processing systems  trained under large-scale weak supervision.  There are nine models of different sizes and capabilities summarized in the following table.  According to the size, the tiny model has 39 million parameters and it is English only.  Base model has a size of 3.5 million parameters.  It has 74 million parameters and it is also English only model.  Then there is a small model, which has around 244 million parameters and it is also English only.  Then there is a medium model.  The medium model has 796 million parameters and it is English o

In [34]:
import time
from pydub import AudioSegment
from faster_whisper import WhisperModel
from jiwer import wer, cer

# model_size = "large-v3"

# # Initialize WhisperModel for fast processing on appropriate hardware
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")

# Define the audio file and split duration
audio_file = "Whisper.wav"
split_duration = 5000  # 5 seconds per chunk, in milliseconds

# Load the audio file correctly
audio = AudioSegment.from_wav(audio_file)

# Reference text for WER and CER calculation
reference_text = "The Whisper models are trained for speech recognition and translation tasks. Capable of transcribing speech audio into the text in the language it is spoken, which is also known as automatic speech recognition, as well as translated into speech, especially in English, which is speech translation. Researchers at OpenAI developed the model to study the robustness of speech processing systems trained under large-scale weak supervision. There are nine models of different sizes and capabilities summarized in the following table. According to the size, the tiny model has 39 million parameters and it is English only. Base model has 74 million parameters and it is also English only model. Then there is a small model, which has around 244 million parameters and it is also English only. Then there is a medium model. The medium model has 796 million parameters and it is English only model as well. Then there is a large model as well, which has 100, sorry, it has 1550 million parameters and it is a multilingual model. Thank you."

# Function to process and transcribe each audio segment
def transcribe_segment(model, segment):
    # Export segment to a temporary audio file - consider using BytesIO instead for in-memory operation
    segment.export("temp_segment.wav", format="wav")
    start_time = time.time()  # Start timing
    segments, info = model.transcribe("temp_segment.wav", beam_size=5)
    end_time = time.time()  # End timing
    latency = end_time - start_time
    return segments, latency

# Prepare to collect results
transcribed_text = ""
total_latency = 0  # Initialize total latency

# Split and transcribe the audio in chunks
for i in range(0, len(audio), split_duration):
    segment = audio[i:i + split_duration]
    transcription_segments, segment_latency = transcribe_segment(model, segment)
    total_latency += segment_latency  # Accumulate latency for each segment
    print(f"Latency for segment: {segment_latency} seconds")
    for transcription in transcription_segments:
        transcribed_text += transcription.text + " "
        print(transcription.text)  # Output the transcription of each segment

# Calculate WER and CER
computed_wer = wer(reference_text, transcribed_text)
computed_cer = cer(reference_text, transcribed_text)

print("\nOverall WER:", computed_wer)
print("Overall CER:", computed_cer)
print("Total transcription latency:", total_latency, "seconds")

Latency for segment: 0.4292299747467041 seconds
 The whisper models are trained for speech recognition and translation.
Latency for segment: 0.46895265579223633 seconds
 Capable of transcribing speech audio into the text
Latency for segment: 0.6677262783050537 seconds
 In the language, it is spoken, which is also known as automatic speech.
Latency for segment: 0.366931676864624 seconds
 recognition as well as translated into speech
Latency for segment: 0.5230717658996582 seconds
 especially in English, which is speech translation.
Latency for segment: 0.380403995513916 seconds
 Researchers at OpenAI developed the model to study the robust
Latency for segment: 0.5705561637878418 seconds
 of speech processing systems trained under large-scale
Latency for segment: 0.48041868209838867 seconds
 supervision. There are nine models of different sizes and capabilities
Latency for segment: 0.4397740364074707 seconds
 summarized in the following table.
Latency for segment: 0.3830268383026123 seco

In [41]:
import torchaudio
from torchaudio.datasets import LIBRISPEECH
from faster_whisper import WhisperModel
from jiwer import wer, cer
from tqdm import tqdm
import time
import csv

# Initialize the Whisper model
model_size = "large-v3"
model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")

# Load the LibriSpeech test dataset
librispeech_test = LIBRISPEECH(root="./data", url="test-clean", download=True)

def transcribe_audio(model, waveform, sample_rate):
    torchaudio.save("temp_audio.wav", waveform, sample_rate)
    start_time = time.time()
    segments, info = model.transcribe("temp_audio.wav", beam_size=5)
    end_time = time.time()
    latency = end_time - start_time
    transcription = ' '.join([segment.text for segment in segments])
    return transcription, latency

# Evaluate on a set number of samples from the dataset
num_samples = 100
results = []

# Setting up CSV file to record the results
with open('transcription_results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sample ID', 'Original Text', 'Transcribed Text', 'WER', 'CER', 'Latency (s)'])

    progress_bar = tqdm(total=num_samples, desc="Processing Transcriptions")
    for i, (waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id) in enumerate(librispeech_test):
        if i >= num_samples:
            break
        transcription, latency = transcribe_audio(model, waveform, sample_rate)
        current_wer = wer(utterance.lower(), transcription.lower())
        current_cer = cer(utterance.lower(), transcription.lower())
        # Writing results to CSV
        writer.writerow([i+1, utterance, transcription, f"{current_wer:.2f}", f"{current_cer:.2f}", f"{latency:.2f}"])
        progress_bar.update(1)
    progress_bar.close()

print("Processing complete. Results saved to 'transcription_results.csv'.")

Processing Transcriptions: 100%|██████████| 100/100 [01:32<00:00,  1.08it/s]

Processing complete. Results saved to 'transcription_results.csv'.
